In [1]:
import pandas as pd
import numpy as np

In [2]:
plays = pd.read_csv('plays.csv')
week1 = pd.read_csv('week1.csv')

In [7]:
previousEvent = 'ball_snap'
weekArray = np.array(week1)
for i, instance in enumerate(weekArray):
    event = instance[8]
    frameId = instance[13]
    if (previousEvent == 'ball_snap' and event != 'ball_snap') or frameId == 1:
        weekArray[i][8] = 'ball_snap'
        previousEvent = 'ball_snap'
    elif (event == 'ball_snap'):
        previousEvent = 'between_snap'
    elif (previousEvent == 'between_snap' and event != 'pass_forward'):
        weekArray[i][8] = 'between_snap'
        previousEvent = 'between_snap'
    elif (event == 'pass_forward'):
        weekArray[i][8] = 'after_thrown'
        previousEvent = 'after_thrown'
    elif (previousEvent == 'after_thrown' and frameId != 1):
        weekArray[i][8] = 'after_thrown'
        previousEvent = 'after_thrown'
        
weekMod = pd.DataFrame(weekArray, columns=week1.columns)
week1['event'] = weekMod['event']
weekMod = week1

Variance

In [9]:
varX = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['x'].agg(['var']).reset_index().rename(columns={"var": "varX"})
varY = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['y'].agg(['var']).reset_index().rename(columns={"var": "varY"})
varS = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['s'].agg(['var']).reset_index().rename(columns={"var": "varS"})

Mean Distance and Variance to nearest opposition

In [10]:
groupedWeek = weekMod.groupby(['gameId', 'playId', 'frameId'])
playerXY = {}
for name, group in groupedWeek:
    playerXY[name] = []
    for row in group.iterrows():
        data = [row[1]['nflId'], row[1]['team'], row[1]['x'], row[1]['y'], row[1]['dir']]
        playerXY[name].append(data)

features = list(weekMod.columns)
weekArray = np.array(weekMod)
minOppDist = []
for player in weekArray:
    if player[features.index('team')] != 'football':
        opponentPositions = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
        distances = []
        directions = []
        opponents = []
        xs = []
        ys = []
        for oppPos in opponentPositions: 
            if player[features.index('team')] != oppPos[1] and player[features.index('team')] != 'football' and oppPos[1] != 'football':
                dx = (player[features.index('x')] - oppPos[2])**2
                dy = (player[features.index('y')] - oppPos[3])**2
                dist = np.sqrt(dx+dy)
                distances.append(dist)
                directions.append(oppPos[4])
                opponents.append(oppPos[0])
                xs.append(oppPos[2])
                ys.append(oppPos[3])
        minDist = min(distances)
        closestOpponent = opponents[np.argmin(distances)]
        opponentDir = directions[np.argmin(distances)]
        opponentX = xs[np.argmin(distances)]
        opponentY = ys[np.argmin(distances)]
        summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestOpponent, opponentDir, opponentX, opponentY]
        minOppDist.append(summary)
        
minOppDist = pd.DataFrame(minOppDist, columns=['gameId', 'playId', 'frameId', 'nflId', 'oppMinDist', 'closestOpp(nflId)', 'oppDir', 'oppX', 'oppY'])
weekMod = pd.merge(weekMod, minOppDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
oppVar = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['oppMinDist'].agg(['var']).reset_index().rename(columns={"var": "oppVar"})
oppMean = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['oppMinDist'].agg(['mean']).reset_index().rename(columns={"mean": "oppMean"})

Mean Distance and Variance to nearest teammate

In [11]:
features = list(weekMod.columns)
weekArray = np.array(weekMod)
minMateDist = []
for player in weekArray:
    if player[features.index('team')] != 'football':
        matePositions = playerXY[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
        distances = []
        mates = []
        xs = []
        ys = []
        for matePos in matePositions: 
            if player[features.index('team')] == matePos[1] and player[features.index('nflId')] != matePos[0] and player[features.index('team')] != 'football' and matePos[1] != 'football':
                dx = (player[features.index('x')] - matePos[2])**2
                dy = (player[features.index('y')] - matePos[3])**2
                dist = np.sqrt(dx+dy)
                distances.append(dist)
                mates.append(matePos[0])
                xs.append(oppPos[2])
                ys.append(oppPos[3])
        minDist = min(distances)
        closestMate = mates[np.argmin(distances)]
        mateX = xs[np.argmin(distances)]
        mateY = ys[np.argmin(distances)]
        summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestMate, mateX, mateY]
        minMateDist.append(summary)
        
minMateDist = pd.DataFrame(minMateDist, columns=['gameId', 'playId', 'frameId', 'nflId', 'mateMinDist', 'closestMate(nflId)', 'mateX', 'mateY'])
weekMod = pd.merge(weekMod, minMateDist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])
mateVar = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['mateMinDist'].agg(['var']).reset_index().rename(columns={"var": "mateVar"})
mateMean = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['mateMinDist'].agg(['mean']).reset_index().rename(columns={"mean": "mateMean"})

Mean and variance of degree of direction of motion

In [12]:
diffDir = np.absolute(weekMod['dir'] - weekMod['oppDir'])
weekMod['diffDir'] = diffDir
oppDirVar = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['diffDir'].agg(['var']).reset_index().rename(columns={"var": "oppDirVar"})
oppDirMean = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['diffDir'].agg(['mean']).reset_index().rename(columns={"mean": "oppDirMean"})

Mean and variance difference in degrees of the direction of motion between the player and the nearest opponent player

In [13]:
ratio = weekMod['oppMinDist'] / np.sqrt((weekMod['oppX'] - weekMod['mateX'])**2 + (weekMod['oppY'] - weekMod['mateY'])**2)
weekMod['oppMateDistRatio'] = ratio
oppMateDistRatioMean = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['oppMateDistRatio'].agg(['mean']).reset_index().rename(columns={"mean": "meanOppMateDistRatio"})
oppMateDistRatioVar = weekMod.groupby(['gameId', 'playId', 'event', 'nflId'])['oppMateDistRatio'].agg(['var']).reset_index().rename(columns={"var": "varOppMateDistRatio"})

Mean and variance ratio of the distance to the nearest opponent player and the distance from the nearest opponent player to the nearest team mate

In [14]:
features = [varX, varY, varS, oppVar, oppMean, mateVar, mateMean, oppDirVar, oppDirMean, oppMateDistRatioMean, oppMateDistRatioVar]
for feature in features:
    weekMod = pd.merge(weekMod, feature, how='left', on=['gameId', 'event', 'playId', 'nflId'])